In [ ]:

from pathlib import Path
import pandas as pd
import joblib
import gradio as gr

BASE = Path.cwd()
OUT = BASE / "output"

CSV_PATH = OUT / "sarpras_sd_prioritas_processed.csv"
MODEL_PATH = OUT / "rf_prioritas_sarpras_sd.joblib"

print("Folder kerja:", BASE.resolve())
print("CSV  :", CSV_PATH.exists(), CSV_PATH)
print("Model:", MODEL_PATH.exists(), MODEL_PATH)

if not CSV_PATH.exists():
    raise FileNotFoundError(
        f"CSV tidak ditemukan: {CSV_PATH}\n"
        "Jalankan dulu notebook training sampai menghasilkan file CSV di folder 'output/'."
    )

if not MODEL_PATH.exists():
    raise FileNotFoundError(
        f"Model tidak ditemukan: {MODEL_PATH}\n"
        "Jalankan dulu notebook training sampai menghasilkan file model di folder 'output/'."
    )

df = pd.read_csv(CSV_PATH)
rf = joblib.load(MODEL_PATH)

drop_cols = {"PROVINSI", "PRIORITY_SCORE", "PRIORITAS_PERBAIKAN_SARPRAS"}
feature_cols = [c for c in df.columns if c not in drop_cols]

if "PROVINSI" not in df.columns:
    raise ValueError("Kolom 'PROVINSI' tidak ada di CSV. Cek file CSV output kamu.")
if not feature_cols:
    raise ValueError("Kolom fitur kosong. Cek struktur CSV output kamu.")

prov_list = sorted(df["PROVINSI"].dropna().unique().tolist())
classes = list(getattr(rf, "classes_", []))

def predict_by_provinsi(provinsi: str):
    row = df.loc[df["PROVINSI"] == provinsi]
    if row.empty:
        return "Provinsi tidak ditemukan di data.", ""

    X_row = row[feature_cols].iloc[[0]]
    pred = rf.predict(X_row)[0]

    if hasattr(rf, "predict_proba") and classes:
        proba = rf.predict_proba(X_row)[0]
        proba_txt = "\n".join([f"{c}: {p:.2f}" for c, p in zip(classes, proba)])
    else:
        proba_txt = "(Model tidak menyediakan probabilitas.)"

    return f"Prediksi: {pred}", proba_txt

demo = gr.Interface(
    fn=predict_by_provinsi,
    inputs=gr.Dropdown(choices=prov_list, label="Pilih Provinsi"),
    outputs=[
        gr.Textbox(label="Hasil Prediksi"),
        gr.Textbox(label="Probabilitas (per kelas)")
    ],
    title="Prediksi Prioritas Perbaikan Sarpras SD (Random Forest)",
    description="Pilih provinsi untuk melihat prediksi prioritas perbaikan sarpras dan probabilitasnya.",
    flagging_mode="never"
)

demo.launch()


c:\Users\Pongo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Folder kerja: D:\file kuliah S5\Applied Machine\TUGAS BESAR APPLIED MACHINE LEARNING
CSV  : True d:\file kuliah S5\Applied Machine\TUGAS BESAR APPLIED MACHINE LEARNING\output\sarpras_sd_prioritas_processed.csv
Model: True d:\file kuliah S5\Applied Machine\TUGAS BESAR APPLIED MACHINE LEARNING\output\rf_prioritas_sarpras_sd.joblib


c:\Users\Pongo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Pongo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
